In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
from decouple import config
import csv
import datetime
import smtplib

f = open('/Users/dreamscape08/Desktop/Osteostrong Project/OSPGRM_files/Lakewood/lakewood-users.json')
lakewood_data = json.load(f)

lakewood_username = config('Lakewood_username')
lakewood_password = config('Lakewood_password')

lakewood_payload = {'username': lakewood_username, 'password': lakewood_password}
loginurl = ('https://osteostrong.org/auth')
lakewood_totals = []


with requests.session() as lakewood_s:
    lakewood_s.post(loginurl, data=lakewood_payload)
    for i in lakewood_data:
        x=i['UserId']['S']
        lakewood_url = f"https://osteostrong.org/chart?UserId={x}&LocationId=00053"
        lakewood_r = lakewood_s.get(lakewood_url)
        lakewood_soup = BeautifulSoup(lakewood_r.content, 'html.parser')
        lakewood_total_sesh = lakewood_soup.find_all('div', attrs={'class': 'col-1'})
        lakewood_Total_Sessions = (lakewood_total_sesh[2].text.strip())
        
        lakewood_sesh_totals = int(lakewood_Total_Sessions)
        lakewood_names = lakewood_soup.find_all('div', attrs={'class': 'col-2'})
        lakewood_name = lakewood_names[2].text.strip()[20:]
        lakewood_totals.append([lakewood_name, lakewood_sesh_totals, x])
        
        
        
# fields = ['Client Name', 'Total Sessions',  'UserId']


def dateme():
    parser = datetime.datetime.now()
    return parser.strftime("%d-%m-%Y")

lakewood_df=pd.DataFrame(lakewood_totals, columns=['Client Name', 'Total Sessions',  'UserId'])
lakewood_df['Client Name']=lakewood_df['Client Name'].str.strip().convert_dtypes(str)

five=lakewood_df[lakewood_df['Total Sessions']==5]
ten = lakewood_df[lakewood_df['Total Sessions']==10]
twenty = lakewood_df[lakewood_df['Total Sessions']==20]
fifty=lakewood_df[lakewood_df['Total Sessions']==50]
seventyfive = lakewood_df[lakewood_df['Total Sessions']==75]
hundo = lakewood_df[lakewood_df['Total Sessions']==100]
hundotwenty = lakewood_df[lakewood_df['Total Sessions']==120]
hundofifty = lakewood_df[lakewood_df['Total Sessions']==150]
toohundo = lakewood_df[lakewood_df['Total Sessions']==200]



alert=pd.concat([five,ten,twenty,fifty,seventyfive,hundo,hundotwenty,hundofifty,toohundo],
                    axis=0,
                    join="outer"
                    ,ignore_index=True).drop(columns=['UserId'])
# alert=alert.drop(columns=['UserId'])
# alert.columns=['Client Name', 'Total Sessions'] 

lakewood_df.to_excel(f'/Users/dreamscape08/Desktop/Osteostrong Project/OS_Operations/lakewood_Client_db_{dateme()}.xlsx', header=True,index=False)

operations_manager = config('opm_email')
sender_proton = config('proton_email')
p_key = config('proton_key')


to = f'{operations_manager}'
admin_email = f'{sender_proton}' # (You should provide your gmail account name)
admin_pass = f'{p_key}' # (You should provide your gmail password)
smtpserver = smtplib.SMTP('localhost',1025)
smtpserver.ehlo()
smtpserver.starttls()
smtpserver.ehlo
smtpserver.login(admin_email, admin_pass)
header = 'To:' + to + '\n' + 'From: ' + admin_email + '\n' + f'Subject:lakewood BASELINE ALERTS-{dateme()} \n'
msg = header + f'\n {alert} \n\n'
print(header)
smtpserver.sendmail(admin_email, to, msg)
print('done!')
smtpserver.close() 


To:josie@osteostrongdenver.com
From: dreamscape08@pm.me
Subject:lakewood BASELINE ALERTS-15-01-2022 

done!
